In [0]:
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import col, when

spark = SparkSession.builder.appName("app").master("local[2]").getOrCreate()

In [0]:
schema = StructType([
    StructField("employee_id",IntegerType(),False),
    StructField("name",StringType(),False),
    StructField("salary",IntegerType(),False)
])
data = [
( 2           , 'Meir'    , 3000)   ,
( 3           , 'Michael' , 3800)   ,
( 7           , 'Addilyn' , 7400)   ,
( 8           , 'Juan'    , 6100)   ,
( 9           , 'Kannon'  , 7700)   
]
emp = spark.createDataFrame(data,schema)
emp.show()

+-----------+-------+------+
|employee_id|   name|salary|
+-----------+-------+------+
|          2|   Meir|  3000|
|          3|Michael|  3800|
|          7|Addilyn|  7400|
|          8|   Juan|  6100|
|          9| Kannon|  7700|
+-----------+-------+------+



In [0]:
# Write a solution to calculate the bonus of each employee. The bonus of an employee is 100% of their salary if the ID of the employee is an odd number and 
# the employee's name does not start with the character 'M'. The bonus of an employee is 0 otherwise.
# Return the result table ordered by employee_id.

emp.withColumn("bonus",when(((col("employee_id")%2==1) & (~col("name").ilike("m%"))),col("salary")).otherwise(0)).select("employee_id","bonus").orderBy("employee_id").show()


+-----------+-----+
|employee_id|bonus|
+-----------+-----+
|          2|    0|
|          3|    0|
|          7| 7400|
|          8|    0|
|          9| 7700|
+-----------+-----+



In [0]:
emp.createOrReplaceTempView("emp")
spark.sql("select employee_id, case when employee_id%2=1 and name not ilike 'm%' then salary else 0 end as bonus from emp order by 1").show()

spark.sql("select employee_id, if(employee_id%2=1 and name not ilike 'm%',salary,0) as bonus from emp order by 1").show()

+-----------+-----+
|employee_id|bonus|
+-----------+-----+
|          2|    0|
|          3|    0|
|          7| 7400|
|          8|    0|
|          9| 7700|
+-----------+-----+

+-----------+-----+
|employee_id|bonus|
+-----------+-----+
|          2|    0|
|          3|    0|
|          7| 7400|
|          8|    0|
|          9| 7700|
+-----------+-----+



In [0]:
spark.stop()